# 15. Classifying Images with Deep Convolutional Neural Networks

In [1]:
%run -i  'watermark.py'

2020-02-12 15:12:18
----------------------
python		3.6.7
----------------------
numpy		1.16.2
scipy		1.1.0
pandas		0.25.1
matplotlib	3.1.1
imageio		2.5.0
----------------------
ipython		7.8.0
----------------------
sklearn		0.20.4
tensorflow	1.13.1
nltk		3.2.4


## 15.01. The building blocks of CNNs

### 15.01.01. Understanding CNNs and features hierachies
### 15.01.02. Performing discrete convolutions
#### 15.01.02.01. Dscrete convolutions in one dimension
#### 15.01.02.02. Padding inputs to control the size of the output feature maps
#### 15.01.02.03. Determining the size of the convolution output

In [2]:
import numpy as np

def convid(x, w, p=0, s=1):
    w_rot = np.array(w[::-1])
    x_padded = np.array(x)
    
    if p > 0:
        zero_pad = np.zeros(shape=p)
        x_padded = np.concatenate([zero_pad, x_padded, zero_pad])
    
    res = []
    
    for i in range(0, int(len(x)/s), s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]] * w_rot))
        
    return np.array(res)

In [3]:
x = [1, 3, 2, 4, 5, 6, 1, 3]
w = [1, 0, 3, 1, 2]

print('Convid implementation:', convid(x, w, p=2, s=1))

Convid implementation: [ 5. 14. 16. 26. 24. 34. 19. 22.]


In [4]:
print('Numpy Results:', np.convolve(x, w, mode='same'))

Numpy Results: [ 5 14 16 26 24 34 19 22]


#### 15.01.02.04. Performing a discrete convoution in 2D

In [5]:
import numpy as np
import scipy.signal

def conv2d(X, W, p=(0,0), s=(1,1)):
    W_rot = np.array(W)[::-1,::-1]
    X_orig = np.array(X)
    n1 = X_orig.shape[0] + 2*p[0]
    n2 = X_orig.shape[1] + 2*p[1]
    X_padded = np.zeros(shape=(n1, n2))
    X_padded[p[0]:p[0]+X_orig.shape[0], 
             p[1]:p[1]+X_orig.shape[1]] = X_orig
    res = []
    
    for i in range(0, int((X_padded.shape[0] - W_rot.shape[0])/s[0])+1, s[0]):
        res.append([])
        
        for j in range(0, int((X_padded.shape[1] - W_rot.shape[1])/s[1])+1, s[1]):
            X_sub = X_padded[i:i+W_rot.shape[0], j:j+W_rot.shape[1]]
            res[-1].append(np.sum(X_sub * W_rot))
    
    return(np.array(res))

In [6]:
X = [[1,3,2,4], [5,6,1,3], [1,2,0,2], [3,4,3,2]]
W = [[1,0,3], [1,2,1], [0,1,1]]

print('Conv2d Impplementation:\n', conv2d(X,W, p=(1,1), s=(1,1)))

Conv2d Impplementation:
 [[11. 25. 32. 13.]
 [19. 25. 24. 13.]
 [13. 28. 25. 17.]
 [11. 17. 14.  9.]]


In [7]:
print('SciPy Results:\n', scipy.signal.convolve2d(X, W, mode='same'))

SciPy Results:
 [[11 25 32 13]
 [19 25 24 13]
 [13 28 25 17]
 [11 17 14  9]]


### 15.01.03. Subsampling layers

## 15.02. Putting everything together - implementing a CNN

### 15.02.01. Working with multiple input or color channels

In [3]:
import tensorflow as tf

img_raw = tf.io.read_file('~/Documents/Figure/graph.jpg')
img = tf.image.decode_image(img_raw)
print('Image shape:', img.shape)

Image shape: <unknown>


In [4]:
import imageio

img = imageio.imread('~/Documents/Figure/graph.jpg')
print('Image shape:', img.shape)

print('Number data type:', img.dtype)

print(img[100:102, 100:102, :])

Image shape: (1563, 1563, 3)
Number data type: uint8
[[[1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]]]


### 15.02.02. Regularizing an NN with dropout

In [2]:
from tensorflow import keras

conv_layer = keras.layers.Conv2D(
    filters=16, 
    kernel_size=(3,3), kernel_regularizer=keras.regularizers.l2(.001))

fc_layer = keras.layers.Dense(
    units=16, 
    kernel_regularizer=keras.regularizers.l2(.001))

### 15.02.03. Loss functions for classification

## 15.03. Implementing a deep CNN using TensorFlow

### 15.03.01. The multilayer CNN architecture
### 15.03.02. Loading and preprocessing the data
### 15.03.03. Implemeting a CNN using the TensorFlow Keras API

#### 15.03.03.01. Configuration CNN layers in Keras
#### 15.03.03.02. Constructing a CNN in Keras

## 15.04. Gender classigcation from face images using

### 15.04.01. Loading the CelebA dataset
### 15.04.02. Image transformation and data augmentation
### 15.04.03. Training a CNN gender classifier

## 15.05. Summary